# lab 3

In [1]:
from enum import Enum

import sqlite3
from enum import Enum
class Country(Enum):
    UA = 'Ukraine'
    USA = 'United States of Amerika'
    CN = 'China'

    @classmethod
    def valueTuples(self):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        """This function does two things. First, creates a l_platforms table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_countrys (country TEXT PRIMARY KEY)')
        # Invoke executemany to insert multiple values at once
        cur.executemany('INSERT INTO l_countrys VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_countrys')
        con.commit()
        
    @classmethod
    def getAll(self):
        """Fetch all records from l_countrys table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_countrys')
        sqlResult = cur.fetchall()
        # Convert tuples in sqlResult to strings and then to enums
        return [Country(c[0]) for c in sqlResult]
con = sqlite3.connect("movies.db")
cur = con.cursor()

class Genre(Enum):
    ACTION = "Action"
    DRAMA = "Drama"
    COMEDY = "Comedy"
    HORROR = "Horror"
    ROMANCE = "Romance"
    SCIFI = "Science Fiction"
    ANIMATION = "Animation"
    THRILLER = "Thriller"
    FANTASY = "Fantasy"
    DOCUMENTARY = "Documentary"
    
    
    @classmethod
    def valueTuples(self):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in Genre]

    @classmethod
    def createTable(self):
        """This function does two things. First, creates a l_genres table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre text)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', Genre.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(self):
        """Fetch all records from l_markets table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        # Convert tuples in sqlResult to strings and then to enums
        return [Genre(p[0]) for p in sqlResult]
    def getMarketsFromString(str):
        """This is a helper method that
        constructs a set out of a comma-separated string
        """
        return {Genre(e) for e in str.split(',')}

Genre.createTable()
print(Genre.getAll)
    
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INT,
                        country TEXT
                        )''')
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country.value))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country.value, self.name))
        con.commit() 

    @classmethod
    def get(self, name):
        cur.execute('SELECT website, year, hq FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        return Director(name, res[0], res[1], Country(res[2]))

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, website, year, hq FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], Country(el[2])) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country.value)
class Movie:
    def __init__(self, title: str, release_date: int, director: Director, genre: Genre, reception: float = None):
        self.title = title
        self.release_date = release_date
        self.director = director
        self.genre = genre
        self.reception = reception

    def __repr__(self):
        return f"{self.title} ({self.release_date}) - Directed by {self.director.name}, Genre: {self.genre.value}, Reception: {self.reception}%"
    
    def __str__ (self):
        rtp = ''
        rtp += (f"""
Title: {self.title}
Release Date: {self.release_date}
Director: {self.director.name}
Genre: {self.genre.value}\n""")
        if self.reception is not None:
            rtp += f"Reception: {self.reception}%"
        else:
            rtp += "Reception: Not available"
        return rtp
    
    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS MOVIES')
        con.commit()
        
    @staticmethod
    def connect():
        conn = sqlite3.connect("movies.db")
        return conn

    @staticmethod
    def create_table():
        conn = Movie.connect()
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS MOVIES
                     (TITLE TEXT, DIRECTOR TEXT, GENRE TEXT, YEAR INTEGER, RATING REAL)''')
        conn.commit()
        conn.close()

    def insert_record(self):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("INSERT INTO MOVIES VALUES (?, ?, ?, ?, ?)", (self.title, self.director.name,self.genre.value , self.release_date,  self.reception))
        conn.commit()
        conn.close()

    @staticmethod
    def select_all():
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("SELECT * FROM MOVIES")
        rows = c.fetchall()
        conn.close()
        return rows

    @staticmethod
    def update_record(title, reception):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("UPDATE MOVIES SET RATING = ? WHERE TITLE = ?", (reception, title))
        conn.commit()
        conn.close()

    @staticmethod
    def delete_record(title):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("DELETE FROM MOVIES WHERE TITLE = ?", (title,))
        conn.commit()
        conn.close()


# Example usage:
director1 = Director("Christopher Nolan", 1234, Country.UA)
director2 = Director("Quentin Tarantino", 2345, Country.UA)

movie1 = Movie("Inception", 2010, director1, Genre.SCIFI, 87)
movie2 = Movie("Pulp Fiction", 1994, director2, Genre.ACTION)

print(movie1)
print()
print(movie2)

<bound method Genre.getAll of <enum 'Genre'>>

Title: Inception
Release Date: 2010
Director: Christopher Nolan
Genre: Science Fiction
Reception: 87%


Title: Pulp Fiction
Release Date: 1994
Director: Quentin Tarantino
Genre: Action
Reception: Not available


# lab 4

In [2]:
#Made changes in lab 3 Movie class
# Creating the MOVIES table if not exists
Movie.removeTable()
Movie.create_table()

# Example usage:
# Inserting records

movie1.insert_record()
movie2.insert_record()

# Selecting all records
print("All records:")
print(Movie.select_all())

# Updating a record
Movie.update_record("Inception", 90)
print("After updating Inception's rating:")
print(Movie.select_all())

# Deleting a record
Movie.delete_record("Pulp Fiction")
print("After deleting Pulp Fiction:")
print(Movie.select_all())

All records:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 87.0), ('Pulp Fiction', 'Quentin Tarantino', 'Action', 1994, None)]
After updating Inception's rating:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 90.0), ('Pulp Fiction', 'Quentin Tarantino', 'Action', 1994, None)]
After deleting Pulp Fiction:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 90.0)]


# lab 5-6


In [3]:
from enum import Enum
class Country(Enum):
    UA = 'Ukraine'
    USA = 'United States of Amerika'
    CN = 'China'

    @classmethod
    def valueTuples(self):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        """This function does two things. First, creates a l_platforms table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_countrys (country TEXT PRIMARY KEY)')
        # Invoke executemany to insert multiple values at once
        cur.executemany('INSERT INTO l_countrys VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_countrys')
        con.commit()
        
    @classmethod
    def getAll(self):
        """Fetch all records from l_countrys table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_countrys')
        sqlResult = cur.fetchall()
        # Convert tuples in sqlResult to strings and then to enums
        return [Country(c[0]) for c in sqlResult]

In [4]:
Country.removeTable()
Country.createTable()

In [5]:
class Movie:
    def __init__(self, title: str, country: Country, release_date: int, director: Director, genre: Genre, reception: float = None):
        self.title = title
        self.release_date = release_date
        self.director = director
        self.genre = genre
        self.reception = reception
        self.country = country

    def __repr__(self):
        return f"{self.title} ({self.release_date}) - Directed by {self.director.name}, Genre: {self.genre.value}, Reception: {self.reception}%, Country: {self.country}"
    
    def __str__ (self):
        rtp = ''
        rtp += (f"""
Title: {self.title}
Release Date: {self.release_date}
Director: {self.director.name}, {self.director.birth_year}, {self.country.value}
Genre: {self.genre.value}\n""")
        if self.reception is not None:
            rtp += f"Reception: {self.reception}%"
        else:
            rtp += "Reception: Not available"
        rtp += f'\nCountry: {self.country.value}' 
        return rtp
    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS MOVIES')
        con.commit()
        
    @staticmethod
    def connect():
        conn = sqlite3.connect("movies.db")
        return conn

    @staticmethod
    def create_table():
        conn = Movie.connect()
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS MOVIES
                     (TITLE TEXT, DIRECTOR TEXT, GENRE TEXT, YEAR INTEGER, RATING REAL, COUNTRY TEXT)''')
        conn.commit()
        conn.close()

    def insert_record(self):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("INSERT INTO MOVIES VALUES (?, ?, ?, ?, ?, ?)", (self.title, self.director.name,self.genre.value , self.release_date,  self.reception, self.country.value))
        conn.commit()
        conn.close()

    @staticmethod
    def select_all():
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("SELECT * FROM MOVIES")
        rows = c.fetchall()
        conn.close()
        return rows

    @staticmethod
    def update_record(title, reception):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("UPDATE MOVIES SET RATING = ? WHERE TITLE = ?", (reception, title))
        conn.commit()
        conn.close()

    @staticmethod
    def delete_record(title):
        conn = Movie.connect()
        c = conn.cursor()
        c.execute("DELETE FROM MOVIES WHERE TITLE = ?", (title,))
        conn.commit()
        conn.close()

    @classmethod
    def get(self, genre):
        """This query uses a LEFT JOIN and an aggregate function string_agg 
        to fetch markets for a phone.
        """
        conn = Movie.connect()
        cur = conn.cursor()
        cur.execute('''SELECT m.TITLE,m.DIRECTOR, m.YEAR, m.RATING, m.COUNTRY FROM MOVIES m
                    LEFT JOIN l_genres g ON m.GENRE = g.genre
                    WHERE m.GENRE = ?''', (genre.value,))
        res = cur.fetchall()
        res = res[0]

        cur.execute('''SELECT d.name,d.birth_year, d.country FROM directors d
                    LEFT JOIN MOVIES m ON d.name = m.DIRECTOR
                    WHERE d.name = ?''', (res[1],))
        res2 = cur.fetchall()
        res2 = res2[0]
        print(res2)
        for mov in res:
            yield Movie(res[0], Country(res[4]), res[2], Director(res2[0],res2[1],Country(res2[2])) ,genre , res[3])

In [6]:
# Example usage:
director1 = Director("Christopher Nolan", 1234, Country.UA)
director2 = Director("Quentin Tarantino", 2345, Country.UA)
movie1 = Movie("Inception",Country.USA, 2010, director1, Genre.SCIFI, 87)
movie2 = Movie("Pulp Fiction", Country.USA, 1994, director2, Genre.ACTION, 90)


In [7]:
#Made changes in lab 3 Movie class
# Creating the MOVIES table if not exists
Movie.removeTable()
Movie.create_table()

# Example usage:
# Inserting records

movie1.insert_record()
movie2.insert_record()

# Selecting all records
print("All records:")
print(Movie.select_all())

# Updating a record
Movie.update_record("Inception", 90)
print("After updating Inception's rating:")
print(Movie.select_all())

# Deleting a record
Movie.delete_record("Pulp Fiction")
print("After deleting Pulp Fiction:")
print(Movie.select_all())

movie2.insert_record()

All records:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 87.0, 'United States of Amerika'), ('Pulp Fiction', 'Quentin Tarantino', 'Action', 1994, 90.0, 'United States of Amerika')]
After updating Inception's rating:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 90.0, 'United States of Amerika'), ('Pulp Fiction', 'Quentin Tarantino', 'Action', 1994, 90.0, 'United States of Amerika')]
After deleting Pulp Fiction:
[('Inception', 'Christopher Nolan', 'Science Fiction', 2010, 90.0, 'United States of Amerika')]


In [8]:
for movie in Movie.get(Genre.SCIFI):
    print(movie)

('Christopher Nolan', 1234, 'Ukraine')

Title: Inception
Release Date: 2010
Director: Christopher Nolan, 1234, United States of Amerika
Genre: Science Fiction
Reception: 90.0%
Country: United States of Amerika

Title: Inception
Release Date: 2010
Director: Christopher Nolan, 1234, United States of Amerika
Genre: Science Fiction
Reception: 90.0%
Country: United States of Amerika

Title: Inception
Release Date: 2010
Director: Christopher Nolan, 1234, United States of Amerika
Genre: Science Fiction
Reception: 90.0%
Country: United States of Amerika

Title: Inception
Release Date: 2010
Director: Christopher Nolan, 1234, United States of Amerika
Genre: Science Fiction
Reception: 90.0%
Country: United States of Amerika

Title: Inception
Release Date: 2010
Director: Christopher Nolan, 1234, United States of Amerika
Genre: Science Fiction
Reception: 90.0%
Country: United States of Amerika


# Lab 7-8?


In [9]:
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INT,
                        country TEXT
                        )''')
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country.value))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country.value, self.name))
        con.commit() 

    @classmethod
    def get(self, name):
        cur.execute('SELECT website, year, hq FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        return Director(name, res[0], res[1], Country(res[2]))

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, website, year, hq FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], Country(el[2])) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country.value)

In [10]:
Director.removeTable()
Director.createTable()
director1.insert()
director2.insert()

```mermaid
erDiagram


    MOVIE {
        TEXT name PK
        TEXT director FK
        TEXT genre FK
        INTEGER year
        REAL rating

    }

    l_countrys {
        TEXT platform PK
    }

    directors {
        TEXT name PK
        TEXT website
        INTEGER year
    }

    l_genres {
        TEXT genre PK
    }



    MOVIE }|--|| directors : "filmed by"
    MOVIE }|--|| l_countrys : "filmed in"
    MOVIE ||--|{ l_genres : "movie genre"
```